In [ ]:
#GNN to predict undirected metabolic graphs
#input: RAG reaction adjacency graph, mapped reaction expression (node features)
#output: edge weights

#include stoichiometric penalties (S*v==0)
#objective function c is known (maximize for biomass accumulation)

In [ ]:
!pip install torch_sparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.2 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.15-cp37-cp37m-linux_x86_64.whl size=516860 sha256=d286ff420d2a6fa6ed0c88efc3bd8d8da97dd0d70192e1fc72f291d9342187ed
  Stored in directory: /root/.cache/pip/wheels/15/68/4d/1414be5c2c622bad35364e13213180797717b6d4b8923936dc
Successfully built torch-sparse


In [ ]:
!pip install torch_scatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=274491 sha256=4cec775d7c686dbf6185f6da0dd42b09097f7e90ceacbc0e5c065fa4eb90c772
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter


In [ ]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 467 kB 5.3 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=1a6e8a7927c6a1671536a854e28534f308501d33cf934f2d6f9ef7d555f7ffaf
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric


In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 5.2 MB/s 
     |████████████████████████████████| 3.5 MB 5.2 MB/s 


In [ ]:
# from torch_geometric.nn import TopKPooling
# from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GINConv
from torch_geometric.nn import MLP
from torch.nn import ReLU
import torch
from torch.optim import Adam
from torch_geometric.data import DataLoader
import torch.nn.functional as F
import pickle
import numpy as np
import pandas as pd 
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import BatchNorm1d
from torch.nn import Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# <u>e. coli core metabolic network</u>
- 72 metabolites
- 95 reactions
- 137 genes

> ## <u>Data Preprocessing

In [ ]:
import pickle as pk
out_path = '/content/drive/MyDrive/METABOLIC_DATA/ecoli_network/'

with open(out_path+ 'linear_solutions/ecoli_res.pickle', 'rb') as b:
    solutions = pk.load(b)

with open(out_path+ 'linear_solutions/ecoli_bounds.pickle', 'rb') as b:
    bounds = pk.load(b)

S_matrix = pd.read_csv(out_path + 'datasets/S_matrix.csv', index_col = 0)

In [ ]:
with open(out_path+ '/linear_solutions/ecoli_graphs.pickle', 'rb') as b:
    graphs = pk.load(b)

In [ ]:
graphs

In [ ]:
S_bool = S_matrix[S_matrix == 0].fillna(1).to_numpy()

$RAG = \hat{S}^T \hat{S}$

 where $\hat{S}$ is boolean S_matrix

In [ ]:
RAG = np.matmul(np.transpose(S_bool), S_bool)
RAG = pd.DataFrame(RAG, index = S_matrix.columns, columns = S_matrix.columns)

In [ ]:
edge_index = np.zeros([1,2])
for i, row in enumerate(RAG.index):
    for f, col in enumerate(RAG.columns):
        if RAG.iloc[i,f] > 0:
            p = ([i,f])
            edge_index = np.vstack([edge_index, p])
edge_index = edge_index[2:,:]

In [ ]:
edge_index = edge_index.reshape(2,2204)
edge_index = torch.from_numpy(edge_index)
edge_index = edge_index.type(torch.LongTensor)



In [ ]:
from torch_geometric.data import Data

In [ ]:
bounds[0].shape

(95, 2)

In [ ]:
# len(list(graphs.keys()))
dictionary = {}
for i, curr_bounds in enumerate(bounds):

    #x = torch.from_numpy(curr_bounds.reshape(95,2)).float() #both upper and lower bounds
    x = torch.from_numpy(curr_bounds[:,0].reshape(95,1)).float() #just upper bound
    y = torch.from_numpy(solutions[i].reshape(95,1)).float()

    data = Data(x=x, y=y, edge_index=edge_index)
    key = "graph_"+str(i)
    dictionary[key] = data

# <u>GNN Implementation

In [ ]:
num_of_nodes = 95
num_of_edge_types = 1

In [ ]:
class GNN(torch.nn.Module):
    def poly_regression():
        #Write the equation here
        pass
    #Too large output_features: Make it 2 or 4.
    def __init__(self, input_features = 1, output_features = 4):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        #Define the layers and activation functions (and pooling if we need to downsample) for the GNN here
        #We can use TopKPooling since it seems to be most efficient theoritically 
        #Can we use GCNConv here? or do we need to create our own message passing network?

        
        # self.conv1 = GCNConv(input_features, output_features) #8 node features
        # self.conv1 = GINConv(Sequential(Linear(input_features, output_features),
        #                BatchNorm1d(output_features), ReLU(),
        #                Linear(output_features, output_features), ReLU()))
        self.conv1 = GINConv(MLP(in_channels=1, hidden_channels=16,
          out_channels=16, num_layers=3))
        # self.pool1 = TopKPooling()
        # self.conv2 = GCNConv(output_features,output_features)
        # self.conv2 = GINConv(Sequential(Linear(output_features, output_features),
        #                BatchNorm1d(output_features), ReLU(),
        #                Linear(output_features, output_features), ReLU()))
        self.conv2 = GINConv(MLP(in_channels=16, hidden_channels=16,
          out_channels=16, num_layers=3))
        # self.pool2 = TopKPooling()

        # self.conv3 = GINConv(Sequential(Linear(output_features, output_features),
        #                BatchNorm1d(output_features), ReLU(),
        #                Linear(output_features, output_features), ReLU()))
        self.conv3 = GINConv(MLP(in_channels=16, hidden_channels=16,
          out_channels=16, num_layers=3))
        # self.conv3 = GCNConv(output_features,2)
        #Add MLP layers after the GCNConv layers for expressive power of GNNs
        # self.MLPLayer1 = ...
        # self.MLPLayer2 = ...
        
        #Node dimensionality would be the reduction in the dimension from the number of node features to whatever dimension
        #Example : if we have 8 node features it could be reduced to 4 then 2
        self.rates = Linear(16, 1)

    def forward(self,data_x,data_edge_index):
        #x = mapped reaction expression - node features
        #Perform forward pass here:
        
        #1. Downsample if it is a large graph (can use TopKPooling)
        # x = self.pool1(x,edge_index)
        
        
        #2. Message passing using the GCNConv - Neighborhood aggregation. 
            #Obtains node embeddings by aggregating information of neighbor nodes.
            #The number of layers corresponds to the number of hops for node aggregation information. 
            #In this case, 2-hop.
        x = self.conv1(data_x,data_edge_index)
        #Maybe try dropout()
        x = x.relu()
        x = self.conv2(x,data_edge_index)
        #Remove this relu()
        x = x.relu()
        x = self.conv3(x,data_edge_index)
        
        
        
        #3. Apply classifier/predictor to classify whether the node is fluxing or not. (2 classes)
            #Can we make this a regression problem for real valued quantities (amount of metabolites)
        # weight = torch.nn.Parameter(torch.FloatTensor(35, 1))
        # score = F.linear(x,weight)
        # score = F.softmax(x, dim=1)
        # score = Linear(35,35)
        output_data = self.rates(x)
        return output_data

        # return x

In [ ]:
#loss function
#enforce S*V = 0
#distance metric of edge weights (true vs. predicted) which captures gene expression information

# def custom_loss(actual, predicted,S):
#     #MSE - Mean Squared Err
#     rmse = mean_squared_error(actual, predicted, squared=False)

torch.set_printoptions(profile="full")

In [ ]:
# print(dictionary)
#Would this training data be of just eh subgraphs we generate?

# def train(train_data):
#     count = 0
#     #for epoch in range(epochs):
#         #Iterate over the training data
#     for key,data in train_data.dataset.items():
#         optimizer.zero_grad()
#         #output would be the classes associated with each node
#         # print(data)

#         output = model(data.x,data.edge_index)
#         # output = output.double()
#         # print("Output data")
#         # print(output)
#         # print(data.y)
#         # print(data.edge_weights.type())
#         #Compute the loss: penalize for output of the model (fluxing or not) versus ground truth 
#         loss = mse_loss(output,data.y) #+ 0.1 * np.sum(output,axis=0)
#         #print("Loss:")
#         #print(loss)
#         #Derive the gradients
#         loss.backward()
#         #Update parameters
#         optimizer.step()
        
#         # print(torch)
#         #Clear gradient
    
#     print(output.shape, data.y.shape)
#     count = 0
#     for i in range(output.shape[0]):

#         if torch.eq(output[i],data.y[i]):
#             count += 1
            
#     print("Loss:")
#     print(loss)
#     print("Accuracy:")
#     print(count, "out of 95")
    # print(count)
    #return output, data.y

In [ ]:
#train function
#loop through the epochs (for hyperparameter tuning)
#Iterate over dataset and compute the loss
model = GNN()
#Change the optimizer to SGD.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#need to use nnl loss due to softmax being the final layer of the gnn
mse_loss = torch.nn.L1Loss()
#Load the data to be trained
training_data = DataLoader(dictionary)

def train(dictionary):
    for key,data in dictionary.items():
        optimizer.zero_grad()  # Clear gradients.
        out = model(data.x, data.edge_index)  # Perform a single forward pass.
        loss = mse_loss(out, data.y)  # Compute the loss solely based on the training nodes.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
    count = 0
    lst = []
    for i in range(out.shape[0]):

        diff = out[i] - data.y[i]
        lst.append(diff)
        # if torch.eq(out[i],data.y[i]):
        #     count += 1
    # print("output - data.y for each data point")
    # print(lst)
    print("Loss:")
    print(loss)
    print("Accuracy:")
    print(count, "out of 95")

In [ ]:
print(training_data.dataset)

{'graph_0': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_1': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_2': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_3': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_4': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_5': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_6': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_7': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_8': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_9': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_10': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_11': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_12': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_13': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_14': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_15': Data(x=[95, 1], edge_index=[2, 2204], y=[95, 1]), 'graph_16': Data(

In [ ]:
train(dictionary)

Loss:
tensor(10.5938, grad_fn=<MseLossBackward0>)


In [ ]:
for epoch in range(0,30):
    train(dictionary)

Loss:
tensor(0.1348, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1314, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1245, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1251, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1281, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1315, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1272, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1313, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1388, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1206, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1158, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1277, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1169, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1319, grad_fn=<L1LossBackward0>)
Accuracy:
0 out of 95
Loss:
tensor(0.1104,

In [ ]:
#test function
testing_data = DataLoader(data, batch_size=batch)
def test():
    #for epoch in range(epochs):
        #Iterate over the training data
    for data in training_data:
        #Perform forward pass
        edge_weight = model(data.x,data.edge_index)
        #Get predictions
        predictions = edge_weight.argmax(dim=1)
        #correct count
        correct += int((predictions == data.edge_index).sum())
    return correct

#Data processing

In [ ]:
import pickle
import pandas as pd
import numpy as np

In [ ]:
with open('/content/drive/MyDrive/metabolic_graphs/1_toy_graphs.pickle', 'rb') as handle:
    graphs= pickle.load(handle)

with open('/content/drive/MyDrive/metabolic_graphs/1_node_features.pickle', 'rb') as handle:
    features= pickle.load(handle)

In [ ]:
features[0]
features

In [ ]:
#edge index RAG
RAG = pd.read_csv("/content/drive/MyDrive/metabolic_graphs/toy_adjacency_matrix.csv",index_col = 0)

In [ ]:
RAG

,R1,R2,R3,R4,R5,R6,R7,R8
R1,1,1,0,0,0,0,0,0
R2,1,2,1,1,0,0,0,0
R3,0,1,2,1,1,0,0,1
R4,0,1,1,2,0,1,1,1
R5,0,0,1,0,2,1,0,2
R6,0,0,0,1,1,2,1,2
R7,0,0,0,1,0,1,1,1
R8,0,0,1,1,2,2,1,3


In [ ]:
edge_index = np.zeros([1,2])
for i, row in enumerate(RAG.index):
    for f, col in enumerate(RAG.columns):
        if RAG.iloc[i,f] > 0:
            p = ([i,f])
            edge_index = np.vstack([edge_index, p])
edge_index = edge_index[2:,:]

In [ ]:
dictionary_features = {}

for j,graph in enumerate(list(graphs.items())):
    edge_features = np.copy(edge_index)
    for i in range(len(edge_features)):
        g = graph[1] + graph[1].transpose() #transpose to make undirected
        edge_features[i] = g[int(edge_index[i][0]),int(edge_index[i][1])]
    
    dictionary_features[j] = edge_features.reshape(2,35)[0]
    

In [ ]:
edge_index

In [ ]:
graphs[4]

array([[0.        , 1.49305904, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.48296056, 1.01009848, 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.1096958 ,
        0.        , 0.        , 0.37326476],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.26356896, 0.        , 0.74652952],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.1096958 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.26356896],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ]])

In [ ]:
dictionary_features[4]

array([1.49305904, 1.49305904, 1.49305904, 1.49305904, 0.        ,
       0.        , 0.48296056, 0.48296056, 1.01009848, 1.01009848,
       0.48296056, 0.48296056, 0.        , 0.        , 0.        ,
       0.        , 0.1096958 , 0.1096958 , 0.37326476, 0.37326476,
       1.01009848, 1.01009848, 0.        , 0.        , 0.        ,
       0.        , 0.26356896, 0.26356896, 0.        , 0.        ,
       0.74652952, 0.74652952, 0.1096958 , 0.1096958 , 0.        ])

In [ ]:
edge_feature_list = list(dictionary_features.values())

In [ ]:
len(list(graphs.keys()))

892

Convert from numpy to torch tensor


In [ ]:
edge_index = edge_index.reshape(2,35)

edge_index = torch.from_numpy(edge_index)

In [ ]:
edge_index = edge_index.type(torch.LongTensor)

In [ ]:
edge_index.dtype

torch.int64

In [ ]:
from torch_geometric.data import Data

Data format: Data(edge_index=[2, num_edges], x=[num_nodes, feature_vector], y=[target_labels/correct_labels])

In [ ]:
# len(list(graphs.keys()))
dictionary = {}
for i in range(len(list(graphs.keys()))):
    x = torch.from_numpy(list(features.values())[i])
    edge_weight = torch.from_numpy(edge_feature_list[i])
    # edge_weight = edge_weight.type(torch.LongTensor)
    data = Data(x=x, edge_index=edge_index,edge_weights = edge_weight)
    key = "graph_"+str(i)
    dictionary[key] = data

In [ ]:
dictionary

#Recurrent GNN
Do we need a hidden state? Can we stick to simple markov process where input state is node states and output is edge weight.

Hidden state z - Amount of transcript

Input state x - Node states

Output y_hat - Edge Weight amount of metabolites.